In [104]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import re
from nltk.util import ngrams

In [105]:

from sentence_transformers import SentenceTransformer
from InstructorEmbedding import INSTRUCTOR
model_1 = INSTRUCTOR('moka-ai/m3e-base')


load INSTRUCTOR_Transformer
max_seq_length  512


In [106]:
def generate_ngrams(row, n):
    
    res= []
    
    if pd.isnull(row['评论内容']) == False:
    
        ngramss = ngrams( re.split('，|。|！|？', row['评论内容']), n)

        for grams in ngramss:
            if grams != '':
                tmp = ''
                for k in range(0, n):
                    tmp = tmp + grams[k] + '，'
                tmp = tmp[:-1]
                res.append(tmp)
        
    return res
            

In [107]:
a = input('请输入你的评论内容：')
df = pd.DataFrame([[a, 1]], columns=['评论内容', 'review_no'])


In [108]:
df['评论内容_new'] = df.apply(generate_ngrams, axis=1,n=1)

df = df.explode('评论内容_new')
df['query_embedding'] = df['评论内容_new'].apply(lambda x: model_1.encode([x]))
query_df = df


In [109]:
query_df

,评论内容,review_no,评论内容_new,query_embedding
0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1,环境优美,"[[0.16429101, 0.56863827, 0.7804627, -0.033979..."
0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1,地理位置好,"[[0.145928, 0.59215295, 0.9692909, 0.014283393..."
0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1,交通方便,"[[-0.05792518, 0.47498274, 0.6477427, -0.52743..."
0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1,房间舒适卫生,"[[-0.3260088, 0.12928548, 0.23302722, -0.75272..."
0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1,服务人员很热情,"[[0.70269626, 0.76538247, 0.66819215, 0.243835..."
0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1,乐于提供各种帮助,"[[0.841296, 0.45207328, 0.5392943, -0.11709887..."
0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1,早餐丰富,"[[0.3494121, 1.0970231, 0.23388791, -0.6668327..."
0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1,设施很人性化,"[[-0.4338992, 0.5178186, 0.9427241, 0.3230781,..."
0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1,网络高速信号好,"[[0.904699, 0.569196, 1.0900158, 0.53058463, 0..."
0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1,窗外风景好,"[[0.48082352, 0.1938777, 0.8320941, -0.1322686..."


In [110]:
# 后缀标着1的，是用moka-ai/m3e-base
# 后缀标着2的，是用thenlper/gte-large-zh
# 后缀标着3的，是用amu/tao-8k


df = pd.read_pickle('faq_df_final.pkl') # 导入FAQ数据，这个是逸燊做的，大概从1000条里面调了两百多条典型的出来当FAQ

sentiment_common = pd.read_excel('商家回复1.xlsx', sheet_name = '通用性回复') # 崇仁编的回复
sentiment_specific = pd.read_excel('商家回复1.xlsx', sheet_name = '针对性回复（new）') # 同上

In [111]:
# 定义一个余弦相似度，用于计算两个语句之间的相似度
def cosine_similarty(a,b):
    res = np.dot(a,b)/(norm(a)*norm(b))
    return res 

In [112]:
# 创建空 DataFrame，方便之后计算
ans_df = pd.DataFrame()

# 对 1000 条评论，循环执行以下内容
for m in range(0, len(query_df)):

    # 如果评论内容非空，其实这个也没啥用
    if query_df['评论内容_new'].iloc[m] != '':
        
        ### huggingface
        #### 获取相似度分数

        # 获取第 m 个评论的向量
        q = query_df['query_embedding'].iloc[m]
        
        #### 在 FAQ 数据中循环计算相似度
        # df 是 FAQ 数据，所以这里获取 FAQ 向量并计算与评论向量的余弦相似度
        df["similarities"] = df['faq_embedding'].apply(lambda x: cosine_similarty(x[0], q[0]))

        # 对相似度进行排序
        cnt_df = df.sort_values("similarities", ascending=False)

        ##### 设置前 5 个相似的计数
        cntcnt_df = cnt_df.head(5)
        min_score = cntcnt_df['similarities'].iloc[4]  # 获取前 5 个相似中最小的相似度
        max_score = cntcnt_df['similarities'].iloc[0]  # 获取前 5 个相似中最大的相似度

        # 对分类结果进行分组并计数
        group_df = cntcnt_df.groupby('分类结果')['faq_new'].count().reset_index(name='cnt')
        group_df = group_df.sort_values("cnt", ascending=False)
        res = group_df.head(1)['分类结果'].iloc[0]  # 获取出现次数最多的分类结果
        ratio_score = group_df.head(1)['cnt'].iloc[0] / 5  # 计算该分类结果占前 5 个相似 FAQ 的比例

        # 将结果存入 ans_df
        ans_df.at[m, 'review_no'] = query_df['review_no'].iloc[m]
        ans_df.at[m, 'query_long'] = query_df['评论内容'].iloc[m]
        ans_df.at[m, 'ngram'] = 1
        ans_df.at[m, 'query'] = query_df['评论内容_new'].iloc[m]
        ans_df.at[m, 'predict'] = res    
        ans_df.at[m, 'minscore'] = min_score  
        ans_df.at[m, 'maxscore'] = max_score          
        ans_df.at[m, 'ratio'] = ratio_score

        



In [113]:
# 对结果进行排序，按照 review_no、query_long 和 ngram 排序
ans_df = ans_df.sort_values(by=['review_no','query_long','ngram'], ascending = [True, False, True])

# 显示前 10 行结果
print(ans_df.head(10))




   review_no                                         query_long  ngram  \
0        1.0  环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...    1.0   
1        1.0  环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...    1.0   
2        1.0  环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...    1.0   
3        1.0  环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...    1.0   
4        1.0  环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...    1.0   
5        1.0  环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...    1.0   
6        1.0  环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...    1.0   
7        1.0  环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...    1.0   
8        1.0  环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...    1.0   
9        1.0  环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...    1.0   

      query       predict  minscore  maxscore  ratio  
0      环境优美    位置-周围环境-优美  0.879752  0.887127    1.0  
1     地理位置好  位置-便利性-交通-发达  0.878517  0.887832    1.0  
2      交通方便  位置-便利性-

In [114]:
# 筛选出满足以下条件的记录：
# best_min: 0.76
# best_max: 0.88
# best_ratio: 0.7
filtered_df = ans_df[(ans_df['minscore'] >= 0.76) & (ans_df['maxscore'] >= 0.88) & (ans_df['ratio'] >= 0.7)]


In [115]:
# 对 filtered_df 按以下字段排序：
# 1. review_no（评论序号），升序
# 2. query_long（原始评论内容），降序
# 3. predict（预测的分类结果），降序
# 4. ratio（比例得分），升序
# 5. maxscore（最大相似度），升序
# 然后对每个组（由 review_no、query_long 和 predict 组合）生成排名（rank），
# 排名从 1 开始，且在每个组内排名是按上面的排序条件倒序排列的。
filtered_df['rank'] = filtered_df.sort_values(by=['review_no','query_long','predict','ratio','maxscore'],
                                               ascending = [True, False, False, True, True]).groupby(['review_no','query_long','predict']).cumcount(ascending=False) + 1
filtered_df = filtered_df[filtered_df['rank']==1]

/var/folders/0h/yp5_0sv92pb6b5x_2748bs380000gn/T/ipykernel_6217/2216456995.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['rank'] = filtered_df.sort_values(by=['review_no','query_long','predict','ratio','maxscore'],


In [116]:
filtered_df

,review_no,query_long,ngram,query,predict,minscore,maxscore,ratio,rank
0,1.0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1.0,环境优美,位置-周围环境-优美,0.879752,0.887127,1.0,1
2,1.0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1.0,交通方便,位置-便利性-交通-发达,0.887403,0.893925,1.0,1
3,1.0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1.0,房间舒适卫生,房间-舒适度-舒适,0.879382,0.898328,1.0,1
4,1.0,环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设...,1.0,服务人员很热情,服务-态度-好,0.891221,0.895538,1.0,1


In [117]:
# 对 filtered_df 按照 'predict' 进行分组，并统计每个 'predict' 分类结果中的唯一 'review_no'（评论序号）的数量
result_df = filtered_df.groupby('predict')['review_no'].nunique().reset_index(name='review_cnt')

# 按照 'review_cnt'（评论数量）降序排序
result_df = result_df.sort_values(['review_cnt'], ascending=False)


In [118]:
sentiment_specific['temp'] = sentiment_specific['preface'] +sentiment_specific['正文']

In [119]:
# 在 filtered_df 当中，我们把1000条评论，拆成了很多词句，然后为每个词句评估了所属的FAQ，这里是把语句的FAQ按照评论内容的序号映射回评论内容，这样一个评论内容就有多个FAQ了 
# 对 filtered_df 按照 'review_no' 进行分组，并获取每个 'review_no' 对应的唯一 'predict' 值
# 结果 DataFrame 的每一行包含一个评论序号和对应的唯一预测结果数组
each_df = filtered_df.groupby('review_no')['predict'].unique().reset_index(name='prediction_result')

# 显示结果 DataFrame each_df
each_df


,review_no,prediction_result
0,1.0,"[位置-周围环境-优美, 位置-便利性-交通-发达, 房间-舒适度-舒适, 服务-态度-好]"


In [120]:
# 定义一个函数，根据每行的预测结果生成相应的回复， 本来KN这里想判断一下的，如果差评到一定地步才用针对性回复，但是我们给暴力解开了，没使用分数，直接给每个分类赋予了一个回答
def match(row):
    # 初始化空字符串，用于存储负面评论
    sentence_neg = ''
    # 初始化一个空列表，用于存储评分
    score_list = []
    res = ''
    # 遍历每个预测结果
    for k in range(0, len(row['prediction_result'])):
        # 获取当前预测结果的分类
        cat = row['prediction_result'][k]
        # 根据分类从 sentiment_specific 数据框中过滤对应的行
        sentiment_df = sentiment_specific[sentiment_specific['分类结果'] == cat]

        # 如果过滤后的 DataFrame 只有一行
        if len(sentiment_df) == 1:
            # 如果该行的 '评论' 列不为空
            if pd.isnull(sentiment_df['temp'].iloc[0]) == False:
                # 将该行的 '评论' 添加到 sentence_neg 中
                sentence_neg = sentence_neg + sentiment_df['temp'].iloc[0]
                # 评分为 -2，表示有针对性回复
                score = -2
                # 将评分添加到 score_list 中
                score_list.append(score)
            else:
                # 根据 '回复类型' 进行评分
                if sentiment_df['回复类型'].iloc[0] == '通用好评':
                    score = 1
                elif sentiment_df['回复类型'].iloc[0] == '通用中性':
                    score = 2
                elif sentiment_df['回复类型'].iloc[0] == '通用差评':
                    score = -1
                
                # 将评分添加到 score_list 中
                score_list.append(score)
    
    # 如果 score_list 不为空
    if len(score_list) > 0:
        # 根据 score_list 中的最小评分生成对应的回复
        if min(score_list) == -2:
            res = sentence_neg
        elif min(score_list) == -1:
            res = sentiment_common['差评'].iloc[0]
        elif min(score_list) == 1:
            res = (sentiment_common['好评'].iloc[0] +
                   sentiment_common['促销'].iloc[0] +
                   sentiment_common['结语'].iloc[0])
        elif min(score_list) == 2:
            res = (sentiment_common['中性评'].iloc[0] +
                   sentiment_common['推广'].iloc[0] +
                   sentiment_common['结语'].iloc[0])
                
    return res

# 对 each_df 应用 match 函数，生成对应的回复并存储在 'response' 列中
each_df['response'] = each_df.apply(match, axis=1)
each_df['response'] = '尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。'+ each_df.apply(match, axis=1) + '期待未来能有机会为您提供更满意的服务，欢迎您再次光临。'

In [128]:
x = []
for i in filtered_df['predict']:
    x.append(i)
print("经分析，识别到以下关键概念："+ str(x))
print("经分析，这条评论对应的回复应为:"+  str(each_df['response']))
# 环境优美，地理位置好，交通方便，房间舒适卫生，服务人员很热情，乐于提供各种帮助，早餐丰富。设施很人性化，网络高速信号好，窗外风景好。有特别的开床服务，房间小摆件的设计也很有特色。性价比不错，下次还要入住

经分析，识别到以下关键概念：['位置-周围环境-优美', '位置-便利性-交通-发达', '房间-舒适度-舒适', '服务-态度-好']
经分析，这条评论对应的回复应为:0    尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。我们很高兴您认为周围环境优美。我们的...
Name: response, dtype: object
